## Input Video

In [1]:
!pip3 install opencv-python

In [2]:
import pandas as pd
import numpy as np # for mathematical operations
from keras.utils import np_utils

import keras
from keras.models import Sequential, load_model
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [3]:
# fine-tuning the model architecture
model = Sequential()
model.add(InputLayer((24*3,)))    # input layer
model.add(Dense(units=10, activation='sigmoid', input_shape=(24*3,))) # hidden layer
model.add(Dense(2, activation='softmax'))    # output layer

model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model = load_model('weight.hdf5')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
# going into folder with training data
path="dataPoints_input/"

# creating a pandas dataframe 
train = pd.DataFrame()

# get name of csv, assuming it's called vids
file = "vid.csv"

train = pd.read_csv(path + file, header=None)

# creating empty dictionary named videoFrames_FILENAME
locals()['videoFrames{}'.format("_" + file.split(".csv")[0])] = {}
# creating empty dictionary named videoCombo_FILENAME
locals()['videoCombo{}'.format("_" + file.split(".csv")[0])] = {}

# number of points per frame
n = 12
    
# create array for each frame and adds them to dictionary
for i in range(len(train) // 12):
    # grouping each frame and making one big array (with 12 x,y points) aka grouping 12 arrays into one array
    data = train.to_numpy()[i * n:(i + 1) * n]
    # deleting frame number from above array
    data = np.delete(data, 0, 1)
    # reshaping each frame array into 24, 1
    data = np.reshape(data, (24, 1))
    # creating tag with name of number of frame and adding above array as value into trainingFrames
    locals()['videoFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = data
        
# goes into each trainingFrame dictionary
for i in locals()['videoFrames{}'.format("_" + file.split(".csv")[0])]:
    # if the point (???) is 
    if int(i) < (len(locals()['videoFrames{}'.format("_" + file.split(".csv")[0])]))-3:
        # counters for one and two above current frame respectively
        j = int(i)+1
        k = int(i)+2
            
        # creating local varriables that store current frame array and the two following
        combine = locals()['videoFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)]
        combineTwo = locals()['videoFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(j)]
        combineThree = locals()['videoFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(k)]
            
        # appending current frame with the following two frames
        combine = np.append(combine, combineTwo, axis=1)
        combine = np.append(combine, combineThree, axis=1)
            
        # creating tag with name of number of frame and adding above combo into trainingCombo
        locals()['videoCombo{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = combine  

# print(videoCombo_vid)

In [5]:
size = len(locals()['videoCombo{}'.format("_" + file.split(".csv")[0])])

pts = []

for i in range(0, size-1):
    val = list(locals()['videoCombo{}'.format("_" + file.split(".csv")[0])]["{}".format(i)])
    pts.append(val)
    
data = np.array(pts)
X = data.reshape(data.shape[0], 24*3)

p = []
nP = [0]

pTags = model.predict_classes(X)
print(pTags)
for i in range(len(pTags)):
    nP[0] = pTags[i]
    p.append(nP[0])
    
# go into predict and calc the percentages of good vs bad ie 0 vs 1
numGood = 0
size = len(p)
for i in range(len(p)):
    numGood = p.count(0)
accuracy = numGood/size * 100
print(accuracy)

# where in video is it bad (1) ?
for i in range(len(p)):
    if p[i] == 1:
        print(i)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
100.0
